# Import Libraries

In [ ]:
import numpy as np
import math
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

# Functions

In [ ]:
def gini_index(X,y,index,split_val):
    
    y[y==-1] = 0
    idx = (X[:,index] < split_val)
    y_left = y[idx]
    size_left = len(y_left)
                
    ridx = (X[:,index] >= split_val)
    y_right = y[ridx]
    size_right = len(y_right)
                
    if size_left==0 or size_right==0:
        return None
    
    samples = X.shape[0]
    size_left = len(y_left)
    size_right = len(y_right)
    y_pred_left = np.zeros(size_left)
    
    left_crt = sum(y_left==y_pred_left)
    left_play = left_crt / size_left
    left_notplay = 1 - left_play
    gini_left = 1 - ( (left_play*left_play) + (left_notplay*left_notplay) )
    
    y_pred_right = np.ones(size_right)
    
    right_crt = sum(y_right==y_pred_right)
    right_play = right_crt / size_right
    right_notplay = 1 - right_play
    gini_right = 1 - ( (right_play*right_play) + (right_notplay*right_notplay))
    
    gini_impurity = (gini_left*(size_left/samples)) + (gini_right*(size_right/samples))
    return gini_impurity


def Adaboost(X, y, rounds):

    """    X : Features of dataset used
           Y : Class Labels of dataset used ([3,5] Here)
      rounds : number of rounds (or) number of weak classifiers  """

    samples, features = np.shape(X)

    #Initialize weights to 1/N
    w = np.full(samples, (1 / samples))

    classifiers = {} #Dictionary to store the set of classifiers
    classifiers['Polarity'], classifiers['Feature_index'], classifiers['Split_value'], classifiers['Alpha'] = [], [], [], []
    #Iterate through rounds
    for _ in range(rounds):

        polarity = 1
        feature_index = None
        split_value = None
        alpha = None

        #Minimum error given for using a certain feature value
        GINI = float('inf')

        #Iterating through all distinct feature values & finding the best split for predicting y
        for index in range(features):
            feature_values = np.expand_dims(X[:, index], axis=1)
            distinct_values = np.unique(feature_values)
            #Try every distinct feature value as a split value
            for split_val in distinct_values:
                p = 1
                pred = np.ones(np.shape(y))
                pred[X[:, index] < split_val] = -1
                cur_gini = sum(w[y != pred])
    
                #erro = gini_index(X,y,index,split_val)
                #if erro==None:
                    #continue
                    
                if cur_gini > 0.5:
                    cur_gini = 1 - cur_gini
                p = -1

                #Saving all values if this split value result in smallest error
                if cur_gini < GINI and cur_gini < 0.5:
                    polarity = p
                    split_value = split_val
                    feature_index = index
                    GINI = cur_gini

        #Calculate alpha to update sample weights
        alpha = 0.5 * math.log((1.0 - GINI) / (GINI + 1e-10))
        #set all predictions to 1 initially
        predictions = np.ones(np.shape(y))
        #Assigning class label as '-1' for those indexes where the sample values are below split value
        negative_idx = (polarity * X[:, feature_index] < polarity * split_value)
        predictions[negative_idx] = -1
        #Calculate new weights
        w = w * np.exp(-alpha * y * predictions)
        w = w / np.sum(w) #Normalize all weights

        #Saving the Classifier
        classifiers['Polarity'].append(polarity)
        classifiers['Feature_index'].append(feature_index)
        classifiers['Split_value'].append(split_value)
        classifiers['Alpha'].append(alpha)
        
    return classifiers


def predict(X,cls):
    
    y_pred = np.zeros((X.shape[0],1))

    #for each classifier, label the samples
    for i in range(len(cls['Polarity'])):
        predictions = np.ones(np.shape(y_pred))
        #Label indexes as '-1' where sample values < split value
        index = (np.array(cls['Polarity'][i]) * X[:, cls['Feature_index'][i]] < np.array(cls['Polarity'][i]) * np.array(cls['Split_value'][i]))
        predictions[index] = -1
        y_pred = y_pred + np.array(cls['Alpha'][i]) * predictions

    y_pred = np.sign(y_pred).flatten()
    return y_pred

# Get Training Data & train the model 

In [ ]:
%%time
# get data
dataset = pd.read_csv("ADA_trainfile.txt", header=None, index_col=False)
test = np.array(pd.read_csv("ADA_testfile.txt", header=None))
data = np.array(dataset)
x = np.array(dataset.iloc[:500,1:])
y = np.array(dataset.iloc[:500,0])

# data contains below digits
digit1 = 3
digit2 = 5

# Change labels to {3 --> -1, 5 --> 1}
y[y == digit1] = -1
y[y == digit2] = 1

# train, test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# Adaboost classification with weak classifiers
rounds = [1,50,100,150,200]
for r in rounds:
    weak_cls = Adaboost(x_train, y_train, r)
    y_pred = predict(x_test,weak_cls)

    accuracy = accuracy_score(y_test, y_pred)
    error = 1 - accuracy
    Train_errors.append(error)
    print("Accuracy for %d classifiers          : %.2f\n" % (r,accuracy))
    print("Training error for %d classifiers    : %.2f\n" % (r,error))
    print("\n")

# Estimating Test error by Adaboost model 

In [ ]:
test_pred = predict(test,weak_cls)
test_pred[test_pred==-1]=3
test_pred[test_pred==1]=5

    
file = open("Boost_50.dat","w",encoding='utf-8')
for i in test_pred:
        file.write(str(i)+'\n')


# Estimating Test error using Sklearn Library

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

test_predicted = clf.predict(test)
test_predicted[ test_predicted == -1] = 3
test_predicted[ test_predicted == 1]  = 5

Tree_accuracy = 0.91 #estimate
Tree_error = np.ones(np.shape(rounds))*(1 - Tree_accuracy) #for plotting graph
print("Estimated Test-set error: %.2f" % (1 - Tree_accuracy))

file = open("ADDTestTree.dat","w",encoding='utf-8')
for i in test_predicted:
    file.write(str(i)+'\n')


# Plotting Graph

In [ ]:
rounds = [1,50,100,150,200]
Train_errors = [0.12,0.74,0.77,0.77,0.85] 
Test_errors = [0.14,0.69,0.69,0.77,0.78] #after all estimations

plt.plot(rounds,Train_errors,'r-',label="Training Error")
plt.plot(rounds,Test_errors,'b',label="Estimated Test error")
plt.plot(rounds,Tree_error,'g-', label="Estimated Test error by Library")
plt.legend()
plt.xlabel('Number of Rounds')
plt.ylabel('Error')
plt.title('Number of Rounds vs Error')